In [5]:
from __future__ import division, print_function
import kf_book.pf_internal as pf_internal
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform
from filterpy.stats import plot_gaussian_pdf
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
from collections import Counter
import pickle
import math as m

In [6]:
from numpy.random import uniform

def create_uniform_particles(x_range, y_range, hdg_range, N):
    particles = np.empty((N, 3))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N)
    particles[:, 2] = uniform(hdg_range[0], hdg_range[1], size=N)
    particles[:, 2] %= 2 * np.pi
    return particles

def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 3))
    particles[:, 0] = mean[0] + (randn(N) * std[0])
    particles[:, 1] = mean[1] + (randn(N) * std[1])
    particles[:, 2] = mean[2] + (randn(N) * std[2])
    particles[:, 2] %= 2 * np.pi
    return particles

In [7]:
#######################
##### Update Step #####
#######################

def update(particles, weights, z, R, beacons):
    for i, beacon in enumerate(beacons):
        distance = np.linalg.norm(particles[:, 0:2] - landmark, axis=1)
        weights *= scipy.stats.norm(distance, R).pdf(z[i])

    weights += 1.e-300      # avoid round-off to zero
    weights /= sum(weights) # normalize

In [8]:
#################################
##### State Estimation Step #####
#################################

def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""

    pos = particles[:, 0:2]
    mean = np.average(pos, weights=weights, axis=0)
    var  = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var

In [9]:
#############################
#### Particle Resampling ####
#############################

def simple_resample(particles, weights):
    N = len(particles)
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1. # avoid round-off error
    indexes = np.searchsorted(cumulative_sum, random(N))

    # resample according to indexes
    particles[:] = particles[indexes]
    weights.fill(1.0 / N)
    
def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights.fill(1.0 / len(weights))    

def neff(weights):
    return 1. / np.sum(np.square(weights))

In [10]:
####################################
#### Transition Matrix Training ####
####################################

def PredLookup(item, index, predlist):
    # print(index)
    # print(predlist[index - 1][1])
    if (predlist[index - 1][1] == item):
        return False;
    else:
        return True;
    

agenttrace = '../4jul11p.csv'
f = open(agenttrace)
#print(f.readlines)
oracle = []
gridized_data = []
gridsize = 0.5

xbins = 13
ybins = 21

# Based on Smart Condo Dimension and orientation:
TMatrix = np.zeros((xbins * ybins, xbins * ybins))
TLookupTable = []

for x in range(xbins):
    for y in range(ybins):
        TLookupTable.append("[" + str(x) + ", " + str(y) + "]")
        
with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            oracle.append([float(row[4]), float(row[5])])
            
for d in oracle:
    gridized_data.append([int(np.floor(d[0]/gridsize)), int(np.floor(d[1]/gridsize))])        
    
for x in range(xbins):
    for y in range(ybins):
        # find (x, y)s in gridized_data
        indices = [index for index, item in enumerate(gridized_data) if item == [x, y] and 
                   PredLookup(item, index, list(zip(gridized_data[1:], gridized_data)))]  
        
        # foreach (x, y): find their next successor; N = total number of successors 
        if not indices:
            print("Agent was not in location [" + str(x) + ", " + str(y) + "] at all")
        else:
            # print([x, y])
            # print(operator.itemgetter(*indices)(gridized_data))
            print(indices)
            # print(gridized_data[for i in indices])
            
            print("shadan")
            N = len(indices)
            next_indices = [x + 1 for x in indices]
            
            next_states = [gridized_data[i] for i in next_indices]
            
            # next_states = [operator.itemgetter(*next_indices)(gridized_data)]
            
            print(next_states)
            
            print("golestan")
            
            for next_state in next_states:
                n = next_states.count(next_state)
                # print(TLookupTable)
                # print(TLookupTable.index(next_state))
                
                cell = TLookupTable.index(str(next_state))
                TMatrix[x, cell] = float(n / N)

Agent was not in location [0, 0] at all
Agent was not in location [0, 1] at all
Agent was not in location [0, 2] at all
Agent was not in location [0, 3] at all
Agent was not in location [0, 4] at all
Agent was not in location [0, 5] at all
Agent was not in location [0, 6] at all
Agent was not in location [0, 7] at all
Agent was not in location [0, 8] at all
Agent was not in location [0, 9] at all
Agent was not in location [0, 10] at all
Agent was not in location [0, 11] at all
Agent was not in location [0, 12] at all
Agent was not in location [0, 13] at all
Agent was not in location [0, 14] at all
Agent was not in location [0, 15] at all
Agent was not in location [0, 16] at all
Agent was not in location [0, 17] at all
Agent was not in location [0, 18] at all
Agent was not in location [0, 19] at all
Agent was not in location [0, 20] at all
Agent was not in location [1, 0] at all
Agent was not in location [1, 1] at all
Agent was not in location [1, 2] at all
Agent was not in location [1,

[39, 80, 5487, 5643]
shadan
[[11, 17], [9, 16], [10, 17], [11, 17]]
golestan
Agent was not in location [10, 18] at all
Agent was not in location [10, 19] at all
Agent was not in location [10, 20] at all
Agent was not in location [11, 0] at all
Agent was not in location [11, 1] at all
Agent was not in location [11, 2] at all
Agent was not in location [11, 3] at all
Agent was not in location [11, 4] at all
Agent was not in location [11, 5] at all
Agent was not in location [11, 6] at all
Agent was not in location [11, 7] at all
Agent was not in location [11, 8] at all
[1136]
shadan
[[10, 9]]
golestan
[1134, 1491, 5508]
shadan
[[11, 10], [11, 10], [11, 10]]
golestan
[1131, 1254]
shadan
[[11, 11], [12, 13]]
golestan
[1063, 1129]
shadan
[[12, 13], [11, 12]]
golestan
Agent was not in location [11, 13] at all
Agent was not in location [11, 14] at all
Agent was not in location [11, 15] at all
Agent was not in location [11, 16] at all
[40, 61, 5484, 5644, 5774, 6024]
shadan
[[11, 17], [11, 17], 

In [159]:
 def transition_matrix(transitions, xbins, ybins):
    states_lookup_table = []
    for x in range(xbins):
        for y in range(ybins):
            states_lookup_table.append("[" + str(x) + ", " + str(y) + "]")

    # print(states_lookup_table)
    n = 1 + len(np.array(list(set(tuple(p) for p in transitions)))) #number of unique states
    
    M = np.ones(((xbins + 2) * (ybins + 2), (xbins + 2) * (ybins + 2)))
    print("Shadan")
    print(xbins + 1)
    print(ybins + 1)
    print(len(M))
    # M = [[0]*n for _ in range(n)]

    for (state, state_p) in zip(transitions,transitions[1:]):
        print("state is: ")
        print(state)
        print("state prime is: ")
        print(state_p)
        i = state[0] * (ybins + 1) + state[1]
        j = state_p[0] * (ybins + 1) + state_p[1]
        print(i)
        print(j)
        # i = state[0]*xbins + state[1]
        # j = state_p[0]*xbins + state_p[1]
        # i = states_lookup_table.index(str(state))
        # j = states_lookup_table.index(str(state_p))
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def beacon_sensor_model():
    filename = '../RSSI.csv'

In [160]:
#test:

agenttrace = '../4jul11p.csv'
f = open(agenttrace)
#print(f.readlines)
oracle = []
gridized_data = []

# Information Granularity
                
gridsize = 0.5
realx = 6.6
realy = 10.6
                
xbins = int(np.ceil(realx) / gridsize) + 2
ybins = int(np.ceil(realy) / gridsize) + 2


with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            oracle.append([float(row[4]), float(row[5])])
            
for d in oracle:
    gridized_data.append([int(np.floor(d[0]/gridsize)), int(np.floor(d[1]/gridsize))])        
           
TM = transition_matrix(gridized_data, xbins, ybins)
                
# for row in m: 
#     print(' '.join('{0:.2f}'.format(x) for x in row))

Shadan
17
25
468
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110
110
state is: 
[4, 10]
state prime is: 
[4, 10]
110

[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 


state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156


state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156


state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[6, 6]
156
156
state is: 
[6, 6]
state prime is: 
[7, 7]
156
182
state is: 
[7, 7]
state prime is: 
[8, 8]
182
208


...

state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 

[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
sta

[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
sta

state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 

31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 
[1, 6]
31
31
state is: 
[1, 6]
state prime is: 

In [169]:
np.max(gridized_data)

20

In [170]:
gridized_data

[[4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [5, 11],
 [6, 12],
 [6, 13],
 [7, 13],
 [8, 14],
 [9, 15],
 [9, 16],
 [10, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [8, 14],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [8, 14],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [11, 17],
 [10, 17],
 [9, 16],
 [8, 15],
 [7, 14],
 [6, 13],
 [5, 13],
 [4, 12],
 [3, 11],
 [2, 10],
 [2, 10],
 [3, 10],
 [3, 10],
 [3, 10],
 [4, 10],
 [4, 10],
 [4, 10],
 [4, 10

In [166]:
ybins

24

In [158]:
(xbins + 2) * (ybins + 2)

468

In [171]:
with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            oracle.append([float(row[4]), float(row[5])])
            
for d in oracle:
    gridized_data.append([int(np.floor(d[0]/gridsize)), int(np.floor(d[1]/gridsize))])        

In [172]:
########################
#### Configurations ####
########################

beacons = np.array([[1, 1], [3, 4.9], [1.35, 7.7], [3.3, 10.5], [3.3, 10.5], [6, 0.5], [0.5, 3.3], [6, 10.5]])
NL = len(beacons)

pik = 'SensorModel.dat'

with open(pik, 'rb') as f:
    beacons_mean = pickle.load(f)
    beacons_stds = pickle.load(f)

In [173]:
AA = np.random.normal(3, 2, 2)

print(AA)

agent_pos = [1, 1]
# for b in beacons:
    # print(round(np.linalg.norm(b - agent_pos)))

[ 3.95989667 -2.16767015]


In [179]:
def CumulativeSum(m):
    return m.cumsum()

def returnobj(cdf):
    # print("aaaaaaaaaaaabbbbbbb:")
    # print(cdf)
    randnum = np.random.random()
    for i in range(len(cdf) - 1):
        if((randnum>cdf[i]) and (randnum<=cdf[i + 1])):
            return i

def IndexToState(index, xbins, ybins):
    # print("aaaaaa:")
    # print(index)
    x = m.floor(index/(ybins + 1))
    y = index % (ybins + 1)
    return x, y

def StateToIndex(x, y, ybins):
    print(x)
    print(y)
    print(ybins)
    return int((ybins + 1) * np.floor(x) + np.floor(y))
        
def NextStates(TM, particles, xbins, ybins):
    cdf = np.apply_along_axis(CumulativeSum, 1, TM)
    zero = np.array([0]) # np.zeros((len(cdf[:,1]))
    
    c = np.tile(zero[np.newaxis,:], (cdf.shape[0],1))
    cdf = np.concatenate((c, cdf), axis=1)
    
    # cdf = np.concatenate([np.zeros((len(cdf[:,1])), 1), cdf])
    next_states = []
    
    # print(cdf)
    
    # print("CDF test 1:")
    # print(cdf[0])
    
    # print("CDF test 2:")
    # print(cdf[:,0])
    
    for i in range(len(particles)):
        current_state_x = particles[i, 0]
        current_state_y = particles[i, 1]
        
        row = StateToIndex(current_state_x, current_state_y, ybins)
        print(len(cdf))
        next_state_index = returnobj(cdf[row])
        next_states.append([IndexToState(next_state_index, xbins, ybins)])
    
    for (state, state_p) in zip(transitions,transitions[1:]):
        i = state[0]*xbins + state[1]
        j = state_p[0]*xbins + state_p[1]
    
    
    return next_states

In [180]:
########################
##### Predict Step #####
########################

def predict(particles, xbins, ybins):
    particles[:, 0] += NextStates(TM, particles, xbins, ybins)[:, 0]
    particles[:, 1] += NextStates(TM, particles, xbins, ybins)[:, 1]
    
    return particles

In [181]:
from filterpy.monte_carlo import systematic_resample
from numpy.linalg import norm
from numpy.random import randn
import scipy.stats

def RssiToDistance(agent_pos):
    rssi_values = []
    for b in beacons:
        dist = round(np.linalg.norm(b - agent_pos))
        
        # Max reading is 7 meters
        if (dist > 7):
            dist = 7
        
        mean = beacons_mean[dist]
        std = beacons_stds[dist]
        
        sample = np.random.normal(mean, std, 1)
        rssi_values.append(sample[0])
        
    return rssi_values

def run_pf1(N, iters=18, sensor_std_err=.1, do_plot=True, plot_particles=False, xlim=(0, 20), ylim=(0, 20), initial_x=None):
    plt.figure()
   
    # create particles and weights
    particles = create_uniform_particles((0,20), (0,20), (0, 6.28), N)    
    weights = np.ones(N) / N

    if plot_particles:
        alpha = .20
        if N > 5000:
            alpha *= np.sqrt(5000)/np.sqrt(N)           
        plt.scatter(particles[:, 0], particles[:, 1], alpha=alpha, color='g')
    
    xs = []
    robot_pos = np.array([0., 0.])
    for gt in range(len(oracle)):
        # Agent's current location
        agent_pos = gt
        
        # move particles based on transition matrix
        # predict(particles, u=(0.00, 1.414), std=(.2, .05))
        particles = predict(particles, xbins, ybins)
        
        # Observation from Beacons' RSSI
        zs =  RssiToDistance(agent_pos, beacons)
        
        # Weighting particles based on our observation 
        update(particles, weights, z = zs, R = sensor_std_err, beacons = beacons)
        
        
    
        
        # resample if too few effective particles
        if neff(weights) < N/2:
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            assert np.allclose(weights, 1/N)
        mu, var = estimate(particles, weights)
        xs.append(mu)

        if plot_particles:
            plt.scatter(particles[:, 0], particles[:, 1], color='k', marker=',', s=1)
        
        p1 = plt.scatter(robot_pos[0], robot_pos[1], marker='+', color='k', s=180, lw=3)
        p2 = plt.scatter(mu[0], mu[1], marker='s', color='r')
    
    xs = np.array(xs)
    #plt.plot(xs[:, 0], xs[:, 1])
    plt.legend([p1, p2], ['Actual', 'PF'], loc=4, numpoints=1)
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    print('final position error, variance:\n\t', mu - np.array([iters, iters]), var)
    plt.show()

In [182]:
from numpy.random import seed
seed(2) 
run_pf1(N=5000, plot_particles=False)

8.719898042840075
11.816946747212285
24
468
0.5185246365578267
13.427229214672675
24
468
10.993249557574183
0.642417159025197
24
468
8.706447852365537
6.639943318044493
24
468
8.40735604174978
14.299492730204705
24
468
6.6066964200774825
13.892364757358084
24
468
4.09297268075685
10.056103939422318
24
468
12.385419327013274
19.472984869853
24
468
5.993093473490463
13.89607149500427
24
468
5.336545502057333
7.273539390068489
24
468
12.422676655385898
8.9666589001196
24
468
10.58284188554078
10.654987346453954
24
468
2.691598906898671
18.043559088293932
24
468
10.27156242531493
19.43468666142828
24
468
3.6887973129383056
12.57130565559984
24
468
15.70670295633347
14.171833474211066
24
468
17.079505852789776
4.697823393456153
24
468
9.884736747638556
7.562740894844802
24
468
16.93122970714936
4.959373493496537
24
468
1.5929095401812199
18.24410842652441
24
468
10.10492180243408
5.493706809666639
24
468
1.3057300877375622
0.8528322599245985
24
468
8.562446551947788
9.244736600019776
24
468

IndexError: index 490 is out of bounds for axis 0 with size 468

<Figure size 432x288 with 0 Axes>

In [178]:
len(cdf)

NameError: name 'cdf' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Our 2-dimensional distribution will be over variables X and Y
N = 100
X = np.linspace(49, 51, N)
Y = np.linspace(49, 51, N)
X, Y = np.meshgrid(X, Y)

# Mean vector and covariance matrix
mu = np.array([50., 50.])
Sigma = np.array([[ 0.2 , 0.], [0.,  0.2]])

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos.

    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.

    """

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, Sigma)
grid = Z
# Create a surface plot and projected filled contour plot under it.
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True, cmap=cm.viridis)

# cset = ax.contourf(X, Y, Z, zdir='z', offset=-0.15, cmap=cm.viridis)

# Adjust the limits, ticks and view angle
# ax.set_zlim(-0.15,0.2)
# ax.set_zticks(np.linspace(0,0.2,5))
ax.view_init(30, 50)

plt.show()

import pylab as pl
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm

# im = plt.matshow(zip(*error_heatmap_averaged), cmap=pl.cm.RdYlGn_r)
im = plt.matshow(grid, cmap=pl.cm.OrRd)

plt.colorbar(im)
plt.show()

In [ ]:
import pylab as pl
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm

# im = plt.matshow(zip(*error_heatmap_averaged), cmap=pl.cm.RdYlGn_r)
im = plt.matshow(grid, cmap=pl.cm.OrRd)

plt.colorbar(im)
plt.show()

In [ ]:
grid

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as sp

x = np.random.randn(100000)
y = np.random.randn(100000)


X = sp.filters.gaussian_filter(x, sigma = 2, order = 0)
Y = sp.filters.gaussian_filter(y, sigma = 2, order = 0)

# normal distribution center at x=0 and y=5
fig1 = plt.subplot(2,2,1)
plt.hist2d(x, y, bins=40)
plt.colorbar()
plt.title('Heatmap without smoothing')
plt.xlabel("X")
plt.ylabel("Y")

In [ ]:
# Add a colorbar if necessary
plt.hexbin(x, y, gridsize=(25,25), cmap=plt.cm.Purples_r)

plt.show()
